In [1]:
import numpy
import time
import pickle
from __future__ import print_function

In [2]:
def add_freq_dict(dictionary, item):
    if item in dictionary:
        dictionary[item] += 1
    else:
        dictionary[item] = 1

        
def load_file(filepath):
    
    with open(filepath, 'r') as f:
        file = f.readlines()
    return file

src_file = load_file('data/file.en')
trg_file = load_file('data/file.de')
aligned_file = load_file('data/file.aligned')

In [25]:
def extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, trg_len):    
  

    if trg_end < 0 :
        return []
    
    a = []
    
    for trg_word, src_word  in alignments:
        if ((trg_start <= trg_word <= trg_end) and 
           (src_word < src_start or src_word > src_end)):  
            return []
        else:
            a.append( (trg_word - trg_start, src_word - src_start) )
            
        
    E = []
    trg_s = trg_start
    
    while True:
        trg_e = trg_end
        while True:
            E.append( (tuple(trg_words[trg_s:trg_e+1]) , tuple(src_words[src_start:src_end+1])) )
            
            trg_e += 1
            
            if trg_e in trg_aligned or trg_e == len(trg_words):
                break 
    
        trg_s -= 1
        
        if trg_s in trg_aligned or trg_s < 0:
            break
    return E, a


def extract_phrases(src_words, trg_words, alignments, cutoff):
    
    
    if cutoff == -1:
        cutoff = len(src_words)
        
        
    trg_aligned = set()
    for (trg,_) in alignments:
        trg_aligned.add(trg)
    
    
    extracted_phrases = []
    for src_start in range(len(src_words)):
        for src_end in range(src_start, min(src_start + cutoff, len(src_words))):

            trg_start = len(trg_words) - 1
            trg_end = - 1
            for  (trg,src) in alignments: 
                if src_start <= src <= src_end:
                    trg_start = min(trg, trg_start)
                    trg_end = max(trg, trg_end)
                    
            
            if(trg_end - trg_start > cutoff - 1):
                continue

            phrase_pairs, a = extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, len(trg_words))
            if (phrase_pairs):
                extracted_phrases.extend(phrase_pairs)
            
    return extracted_phrases

('asd', 'asdsd')

In [26]:
# src_words = "michael assumes that he will stay in the house".split()
# trg_words = "michael geht davon aus , dass er im haus bleibt".split()
# alignments = [(0,0), (1,1), (2,1), (3,1), (5,2), (6,3), (9,4), (9,5), (7,6), (7,7), (8,8)]
 
# BP = extract_phrases(src_words, trg_words, alignments)
    
# freq_pairs = dict()
# freq_src = dict()
# freq_trg = dict()
    
# for pair in BP:
#     add_freq_dict(freq_pairs,pair)
#     add_freq_dict(freq_trg, pair[0])
#     add_freq_dict(freq_src, pair[1])

In [27]:
freq_pairs = dict()
freq_src = dict()
freq_trg = dict()
    
start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()]
    
    BP = extract_phrases(src_words, trg_words, alignments, 5)
    
    for pair in BP:
        add_freq_dict(freq_pairs,pair)
        add_freq_dict(freq_trg, pair[0])
        add_freq_dict(freq_src, pair[1])
        
    print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

print()
print('Done!!!')
print('Total duration: %ds'%(time.time() - start_time))

print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))

50000 / 50000 (135s)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
Done!!!
Total duration: 135s
#en phrases: 1189230
#de phrases: 1386442
#(en,de) phrases: 2624229


In [28]:
for key in freq_pairs.keys()[:10]:
    print(key, freq_pairs[key])

(('ausgegeben', ',', 'um', 'die', 'natur', 'zu', 'sch\xc3\xa4digen', '.'), ('to', 'damage', 'the', 'countryside', '.')) 1
(('rede', ',', 'sondern', 'auch', 'von', 'rechten'), ('but', 'also', 'of', 'rights')) 1
(('gef\xc3\xa4hrlichen', 'autowracks', 'verschandelt', 'werden', ','), ('dangerous', 'wrecks', ',', 'inhabited')) 1
(('der', 'kriminalit\xc3\xa4t', 'an', 'die', 'hand'), ('crime',)) 1
(('position', 'bezogen', '.'), ('.',)) 1
(('bev\xc3\xb6lkerung', 'sind', 'analphabeten'), ('africans', 'are', 'illiterate')) 1
(('was', 'sind'), ('what', ',')) 1
(('handeln', 'bereit', 'sein'), ('be', 'prepared', 'to', 'act')) 1
(('0265', '/', '1999', '-', '1999'), ('0265', '/', '1999', '-', '1999')) 3
(('wir', 'wollen', 'daran'), ('so',)) 1


In [ ]:
freq_pairs = dict()
freq_src = dict()
freq_trg = dict()
    
start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()]
    
    BP = extract_phrases(src_words, trg_words, alignments)
    
    for pair in BP:
        add_freq_dict(freq_pairs,pair)
        add_freq_dict(freq_trg, pair[0])
        add_freq_dict(freq_src, pair[1])
        
    print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

print()
print('Done!!!')
print('Total duration: %ds'%(time.time() - start_time))

print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))

In [ ]:
#Save files
# with open("data/freq_src_small", 'wb') as file:
#     pickle.dump(freq_src, file)

# with open("data/freq_trg_small", 'wb') as file:
#     pickle.dump(freq_trg, file)

# with open("data/freq_pairs_small", 'wb') as file:
#     pickle.dump(freq_pairs, file)

with open("data/joined_freq_src", 'rb') as file:
    freq_src = pickle.load(file)

with open("data/joined_freq_trg", 'rb') as file:
    freq_trg = pickle.load(file)

with open("data/joined_freq_pairs", 'rb') as file:
    freq_pairs = pickle.load(file)
        
print("#en phrases: %d"%(len(freq_src)))
print("#de phrases: %d"%(len(freq_trg)))
print("#(en,de) phrases: %d"%(len(freq_pairs)))     

In [ ]:
# import operator
# sorted_freq_src = sorted(freq_src.items(), key=operator.itemgetter(1), reverse = True)
# sorted_freq_trg = sorted(freq_trg.items(), key=operator.itemgetter(1), reverse = True)
# sorted_freq_pairs = sorted(freq_pairs.items(), key=operator.itemgetter(1), reverse = True)

In [ ]:
src_total = 0
for _, freq in freq_src.iteritems():
    src_total += freq
    
trg_total = 0
for _, freq in freq_trg.iteritems():
    trg_total += freq
    
pairs_total = 0
for _, freq in freq_pairs.iteritems():
    pairs_total += freq

print("#en phrases: %d"%(src_total))
print("#de phrases: %d"%(trg_total))
print("#(en,de) phrases: %d"%(pairs_total))     

In [ ]:
for phrase_pair, freq in freq_pairs.iteritems():
    p_src_trg = 1.0 * freq / freq_trg[phrase_pair[0]]
    p_trg_src = 1.0 * freq / freq_src[phrase_pair[1]]
    
    freq_pairs[phrase_pair] = [freq, p_src_trg, p_trg_src]
    
    
    

In [ ]:
keys = freq_pairs.keys()[:5]

for key in keys:
    print(key[0], '|' , key[1], ':', freq_pairs[key])
    print(freq_trg[key[0]])
    print(freq_src[key[1]])


In [ ]:

with open("data/joined_freq_pairs_all", 'wb') as file:
    pickle.dump(freq_pairs, file)